In [ ]:
#read in the data
import pandas as pd
import numpy as np


gender_meta = pd.read_csv('gender_df.csv')

In [ ]:
df_all = pd.read_csv('df_all_fiction.csv')

In [ ]:
#apply gender dict
gender_dict = pd.Series(gender_meta['gender'].values, index=gender_meta['author']).to_dict()
df_all['gender'] = df_all['creator'].map(gender_dict).fillna('unknown')
#
# gender_dict = pd.Series(gender_meta['hand_gender'].values, index=gender_meta['author']).to_dict()
# df_all['hand_gender'] = df_all['creator'].map(gender_dict).fillna('unknown')


In [ ]:
df_all['log_citations'] = np.log1p(df_all['citationCount'])

In [ ]:
# Citations model
import statsmodels.formula.api as smf

model_cite = smf.ols(
    formula='log_citations ~  C(gender) * precocity + year',
    data=df_all
).fit()

print(model_cite.summary())